### 🛠️ Initialize Notebook Variables

This Azure Maps APIM sample demonstrates backend integration and authentication patterns with Azure Maps APIs.

**Configuration required:**
- Review and modify the parameters in the initialization cell below
- Ensure you have appropriate Azure Maps resources available for testing

In [ ]:
import utils
from apimtypes import *

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

rg_location = 'eastus2'
index       = 1
apim_sku    = APIM_SKU.BASICV2              # Options: 'BASICV2', 'STANDARDV2', 'PREMIUMV2'
deployment  = INFRASTRUCTURE.SIMPLE_APIM    # Options: 'AFD_APIM_PE', 'APIM_ACA', 'SIMPLE_APIM'
api_prefix  = ''            # using a prefix results in some issues with at least the default API GET. It's fine to leave it off.
tags        = ['azure-maps']



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

sample_folder    = 'azure-maps'
rg_name          = utils.get_infra_rg_name(deployment, index)
supported_infras = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM]
nb_helper        = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, supported_infras, index = index, apim_sku = apim_sku)
azure_maps_url   = 'https://atlas.microsoft.com'

# Define the APIs and their operations and policies

# Define named values
nvs: List[NamedValue] = [
    NamedValue('azure-maps-arm-api-version', '2023-06-01')
]

# Load policy definitions
pol_map_async_geocode_batch_v1_keyauth_post = utils.read_policy_xml('map_async_geocode_batch_v1_keyauth_post.xml', sample_name=sample_folder)
pol_map_default_route_v2_aad_get            = utils.read_policy_xml('map_default_route_v2_aad_get.xml', sample_name=sample_folder)
pol_map_geocode_v2_aad_get                  = utils.read_policy_xml('map_geocode_v2_aad_get.xml', sample_name=sample_folder)

# API 1: Maps
map_path              = f'{api_prefix}map'      
mapApi_v2_default_get = GET_APIOperation2('get-default-route', 'Get default route', '/default/*', 'This is the default route that will allow all requests to go through to the backend api', pol_map_default_route_v2_aad_get)
mapApi_v1_async_post  = APIOperation('async-geocode-batch', 'Async Geocode Batch', '/geocode/batch/async', HTTP_VERB.POST, 'Post geocode batch async endpoint', pol_map_async_geocode_batch_v1_keyauth_post)
mapApi_v2_geocode_get = GET_APIOperation2('get-geocode', 'Get Geocode', '/geocode', 'Get geocode endpoint', pol_map_geocode_v2_aad_get)

maps = API(map_path, 'Map API', map_path, 'This is the proxy for Azure Maps', 
           operations = [mapApi_v2_default_get, mapApi_v1_async_post, mapApi_v2_geocode_get], tags = tags, serviceUrl = azure_maps_url)

# APIs Array
apis: List[API] = [maps]

utils.print_ok('Notebook initialized')

### 🚀 Deploy Infrastructure and APIs

Creates the bicep deployment into the previously-specified resource group. A bicep parameters, `params.json`, file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_apis        = output.getJson('apiOutputs', 'APIs')

    utils.print_ok('Deployment completed successfully')
else:
    utils.print_error("Deployment failed!")
    raise SystemExit(1)

### ✅ Verify Deployment

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimtesting import ApimTesting
from apimrequests import ApimRequests
import json

# Initialize testing framework
tests = ApimTesting("Azure Maps Sample Tests", sample_folder, deployment)
api_subscription_key = apim_apis[0]['subscriptionPrimaryKey']

# Get the appropriate endpoint URL for testing
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)
reqs = ApimRequests(endpoint_url, api_subscription_key)

# Test Azure Maps API endpoints
utils.print_info("Testing Azure Maps API operations...")

# Test default route with SAS token auth
output = reqs.singleGet(f'{map_path}/default/geocode?query=15127%20NE%2024th%20Street%20Redmond%20WA', msg = 'Calling Default Route API with SAS Token Auth. Expect 200.')
tests.verify('address' in output, True)

# Test geocode v2 with AAD auth
output = reqs.singleGet(f'{map_path}/geocode?query=15127%20NE%2024th%20Street%20Redmond%20WA', msg = 'Calling Geocode v2 API with AAD Auth. Expect 200.')
tests.verify('address' in output, True)

# Test async geocode batch with shared key auth
output = reqs.singlePostAsync(f'{map_path}/geocode/batch/async', data={
    "batchItems": [
        {"query": "?query=400 Broad St, Seattle, WA 98109&limit=3"},
        {"query": "?query=One, Microsoft Way, Redmond, WA 98052&limit=3"},
        {"query": "?query=350 5th Ave, New York, NY 10118&limit=1"},
        {"query": "?query=Pike Pl, Seattle, WA 98101&lat=47.610970&lon=-122.342469&radius=1000"},
        {"query": "?query=Champ de Mars, 5 Avenue Anatole France, 75007 Paris, France&limit=1"}
    ]
}, msg = 'Calling Async Geocode Batch v1 API with Share Key Auth. Expect initial 202, then a 200 on the polling response', timeout = 120, poll_interval = 3)

# Verify batch response contains successful requests
tests.verify('summary' in output and 'successfulRequests' in output and 
            json.loads(output)['summary']['successfulRequests'] == 5, True)

# Test unauthorized access (should fail with 401)
reqsNoApiSubscription = ApimRequests(endpoint_url)
output = reqsNoApiSubscription.singleGet(f'{map_path}/geocode?query=15127%20NE%2024th%20Street%20Redmond%20WA', 
                                        msg='Calling Geocode v2 API without API subscription key. Expect 401.')
outputJson = utils.get_json(output)
tests.verify(outputJson['statusCode'], 401)
tests.verify(outputJson['message'], 'Access denied due to missing subscription key. Make sure to include subscription key when making requests to an API.')

tests.print_summary()
utils.print_ok('✅ All tests completed successfully!')